## Tutorial: Creating a Machine Learning Transform with AWS Glue

This tutorial guides you through the actions to create and manage a machine learning (ML) transform using AWS Glue. Before using this tutorial, you should be familiar with using the AWS Glue console to add crawlers and jobs and edit scripts. You should also be familiar with finding and downloading files on the Amazon Simple Storage Service (Amazon S3) console.

In this example, you create a `FindMatches` transform to find matching records, teach it how to identify matching and nonmatching records, and use it in an AWS Glue job. The AWS Glue job writes a new Amazon S3 file with an additional column named `match_id`.

The source data used by this tutorial is a file named `dblp_acm_records.csv`. This file is a modified version of academic publications (DBLP and ACM) available from the original [DBLP ACM dataset](https://doi.org/10.3886/E100843V2). The `dblp_acm_records.csv` file is a comma-separated values (CSV) file in UTF-8 format with no byte-order mark (BOM).

A second file, `dblp_acm_labels.csv`, is an example labeling file that contains both matching and nonmatching records used to teach the transform as part of the tutorial.

Topics

- [__Step 1: Crawl the Source Data__](https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html#ml-transform-tutorial-crawler)
- [__Step 2: Add a Machine Learning Transform__](https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html#ml-transform-tutorial-create)
- [__Step 3: Teach Your Machine Learning Transform__](https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html#ml-transform-tutorial-teach)
- [__Step 4: Estimate the Quality of Your Machine Learning Transform__](https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html#ml-transform-tutorial-estimate-quality)
- [__Step 5: Add and Run a Job with Your Machine Learning Transform__](https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html#ml-transform-tutorial-add-job)
- [__Step 6: Verify Output Data from Amazon S3__](https://docs.aws.amazon.com/glue/latest/dg/machine-learning-transform-tutorial.html#ml-transform-tutorial-data-output)

### Step 1: Crawl the Source Data
First, crawl the source Amazon S3 CSV file to create a corresponding metadata table in the Data Catalog.

__Important__
To direct the crawler to create a table for only the CSV file, store the CSV source data in a different Amazon S3 folder from other files.

1. Sign in to the AWS Management Console and open the AWS Glue console at https://console.aws.amazon.com/glue/.

2. In the navigation pane, choose __Crawlers, Add crawler__.

3. Follow the wizard to create and run a crawler named `demo-crawl-dblp-acm` with output to `database demo-db-dblp-acm`. When running the wizard, create the database `demo-db-dblp-acm` if it doesn't already exist. Choose an Amazon S3 include path to sample data in the current AWS Region. For example, for us-east-1, the Amazon S3 include path to the source file is `s3://ml-transforms-public-datasets-us-east-1/dblp-acm/records/dblp_acm_records.csv`.

If successful, the crawler creates the table `dblp_acm_records_csv` with the following columns: id, title, authors, venue, year, and source.

### Step 2: Add a Machine Learning Transform
Next, add a machine learning transform that is based on the schema of your data source table created by the crawler named `demo-crawl-dblp-acm`.

1. On the AWS Glue console, in the navigation pane, choose __ML Transforms, Add transform__. Then follow the wizard to create a `Find matches` transform with the following properties.

    a. For __Transform name__, enter `demo-xform-dblp-acm`. This is the name of the transform that is used to find matches in the source data.

    b. For __IAM role__, choose an IAM role that has permission to the Amazon S3 source data, labeling file, and AWS Glue API operations. For more information, see Create an IAM Role for AWS Glue in the AWS Glue Developer Guide.

    c. For __Data source__, choose the table named __dblp_acm_records_csv__ in database __demo-db-dblp-acm__.

    d. For __Primary key__, choose the primary key column for the table, __id__.

    e. For __Predictive columns__, choose the __title, authors, venue, year, and source columns__ in the data source to act as predictive columns.

In the wizard, choose __Finish__ and return to the __ML transforms__ list.

### Step 3: Teach Your Machine Learning Transform
Next, you teach your machine learning transform using the tutorial sample labeling file.

You can't use a machine language transform in an extract, transform, and load (ETL) job until its status is __Ready for use__. To get your transform ready, you must teach it how to identify matching and nonmatching records by providing examples of matching and nonmatching records. To teach your transform, you can __Generate a label file__, __add labels__, and then __Upload label file__. In this tutorial, you can use the example labeling file named `dblp_acm_labels.csv`. For more information about the labeling process, see [Labeling](https://docs.aws.amazon.com/glue/latest/dg/machine-learning.html#machine-learning-labeling).

1. On the AWS Glue console, in the navigation pane, choose __ML Transforms__.

2. Choose the `demo-xform-dblp-acm` transform, and then choose __Action, Teach__. Follow the wizard to teach your `Find matches` transform.

3. On the transform properties page, choose __I have labels__. Choose an Amazon S3 path to the sample labeling file in the current AWS Region. For example, for `us-east-1`, upload the provided labeling file from the Amazon S3 path `s3://ml-transforms-public-datasets-us-east-1/dblp-acm/labels/dblp_acm_labels.csv` with the option to __overwrite__ existing labels. The labeling file must be located in Amazon S3 in the same Region as the AWS Glue console.

When you upload a labeling file, a task is started in AWS Glue to add or overwrite the labels used to teach the transform how to process the data source.

4. On the final page of the wizard, choose __Finish__, and return to the __ML transforms__ list.

### Step 4: Estimate the Quality of Your Machine Learning Transform
Next, you can estimate the quality of your machine learning transform. The quality depends on how much labeling you have done. For more information about estimating quality, see [Estimate quality](https://docs.aws.amazon.com/glue/latest/dg/console-machine-learning-transforms.html#console-machine-learning-transforms-metrics).

1. On the AWS Glue console, in the navigation pane, choose __ML Transforms__.

2. Choose the `demo-xform-dblp-acm transform`, and choose the __Estimate quality__ tab. This tab displays the current quality estimates, if available, for the transform.

3. Choose __Estimate quality__ to start a task to estimate the quality of the transform. The accuracy of the quality estimate is based on the labeling of the source data.

4. Navigate to the __History__ tab. In this pane, task runs are listed for the transform, including the __Estimating quality__ task. For more details about the run, choose __Logs__. Check that the run status is __Succeeded__ when it finishes.

### Step 5: Add and Run a Job with Your Machine Learning Transform
In this step, you use your machine learning transform to add and run a job in AWS Glue. When the transform `demo-xform-dblp-acm` is __Ready for use__, you can use it in an ETL job.

1. On the AWS Glue console, in the navigation pane, choose __Jobs__.

2. Choose __Add job__, and follow the steps in the wizard to create an ETL Spark job with a generated script. Choose the following property values for your transform:

    a. For __Name__, choose the example job in this tutorial, __demo-etl-dblp-acm__.

    b. For __IAM role__, choose an IAM role with permission to the Amazon S3 source data, labeling file, and AWS Glue API operations. For more information, see [Create an IAM Role for AWS Glue](https://docs.aws.amazon.com/glue/latest/dg/create-an-iam-role.html) in the AWS Glue Developer Guide.

    c. For __ETL language__, choose __Scala__. This is the programming language in the ETL script.

    d. For __Script file name__, choose __demo-etl-dblp-acm__. This is the file name of the Scala script (same as the job name).

    e. For __Data source__, choose __dblp_acm_records_csv__. The data source you choose must match the machine learning transform data source schema.

    f. For __Transform type__, choose __Find matching records__ to create a job using a machine learning transform.

    g. Clear __Remove duplicate records__. You don't want to remove duplicate records because the output records written have an additional match_id field added.

    h. For __Transform__, __choose demo-xform-dblp-acm__, the machine learning transform used by the job.

    i. For __Create tables__ in your data target, choose to create tables with the following properties:

        - __Data store type__ : Amazon S3

        - __Format__ : CSV

        - __Compression type__ : None

        - __Target path__ : The Amazon S3 path where the output of the job is written (in the current console AWS Region)

3. Choose __Save job and edit script__ to display the script editor page.

4. Edit the script to add a statement to cause the job output to the __Target path__ to be written to a single partition file. Add this statement immediately following the statement that runs the `FindMatches` transform. The statement is similar to the following.
    ```scala
    val single_partition = findmatches1.repartition(1) 
    ```

You must modify the .writeDynamicFrame(findmatches1) statement to write the output as .writeDynamicFrame(single_partion).

5. After you edit the script, choose Save. The modified script looks similar to the following code, but customized for your environment.

```scala
import com.amazonaws.services.glue.GlueContext
import com.amazonaws.services.glue.errors.CallSite
import com.amazonaws.services.glue.ml.FindMatches
import com.amazonaws.services.glue.util.GlueArgParser
import com.amazonaws.services.glue.util.Job
import com.amazonaws.services.glue.util.JsonOptions
import org.apache.spark.SparkContext
import scala.collection.JavaConverters._

object GlueApp {
  def main(sysArgs: Array[String]) {
    val spark: SparkContext = new SparkContext()
    val glueContext: GlueContext = new GlueContext(spark)
    // @params: [JOB_NAME]
    val args = GlueArgParser.getResolvedOptions(sysArgs, Seq("JOB_NAME").toArray)
    Job.init(args("JOB_NAME"), glueContext, args.asJava)
    // @type: DataSource
    // @args: [database = "demo-db-dblp-acm", table_name = "dblp_acm_records_csv", transformation_ctx = "datasource0"]
    // @return: datasource0
    // @inputs: []
    val datasource0 = glueContext.getCatalogSource(database = "demo-db-dblp-acm", tableName = "dblp_acm_records_csv", redshiftTmpDir = "", transformationContext = "datasource0").getDynamicFrame()
    // @type: FindMatches
    // @args: [transformId = "tfm-123456789012", emitFusion = false, survivorComparisonField = "<primary_id>", transformation_ctx = "findmatches1"]
    // @return: findmatches1
    // @inputs: [frame = datasource0]
    val findmatches1 = FindMatches.apply(frame = datasource0, transformId = "tfm-123456789012", transformationContext = "findmatches1")
  
  
    // Repartition the previous DynamicFrame into a single partition. 
    val single_partition = findmatches1.repartition(1)    
 
    
    // @type: DataSink
    // @args: [connection_type = "s3", connection_options = {"path": "s3://aws-glue-ml-transforms-data/sal"}, format = "csv", transformation_ctx = "datasink2"]
    // @return: datasink2
    // @inputs: [frame = findmatches1]
    val datasink2 = glueContext.getSinkWithFormat(connectionType = "s3", options = JsonOptions("""{"path": "s3://aws-glue-ml-transforms-data/sal"}"""), transformationContext = "datasink2", format = "csv").writeDynamicFrame(single_partition)
    Job.commit()
  }
}
```

6. Choose __Run job__ to start the job run. Check the status of the job in the jobs list. When the job finishes, in the __ML transform, History tab__, there is a new __Run ID__ row added of type __ETL job__.

7. Navigate to the __Jobs, History__ tab. In this pane, __job runs__ are listed. For more details about the __run, choose Logs__. Check that the run status is __Succeeded__ when it finishes.

### Step 6: Verify Output Data from Amazon S3
In this step, you check the output of the job run in the Amazon S3 bucket that you chose when you added the job. You can download the output file to your local machine and verify that matching records were identified.

1. Open the Amazon S3 console at https://console.aws.amazon.com/s3/.

2. Download the target output file of the job demo-etl-dblp-acm. Open the file in a spreadsheet application (you might need to add a file extension .csv for the file to properly open).

The following image shows an excerpt of the output in Microsoft Excel.

<img src="https://docs.aws.amazon.com/glue/latest/dg/images/demo_output_dblp_acm.png" align="left" alt="excel image" width = "800">

The data source and target file both have 4,911 records. However, the `Find matches` transform adds another column named `match_id` to identify matching records in the output. Rows with the same `match_id` are considered matching records.

3. Sort the output file by `match_id` to easily see which records are matches. Compare the values in the other columns to see if you agree with the results of the `Find matches` transform. If you don't, you can continue to teach the transform by adding more labels.

You can also sort the file by another field, such as `title`, to see if records with similar titles have the same `match_id`.